# Zoom Consistency Report (One Excel • Sheets = Months)

This notebook reads your **Insights - Zoom.xlsx** (or **Insights - Zoom - Marketing.xlsx**) and builds a single Excel file where **each sheet is a month** you select.  
Within each month sheet, the report lists **all anchors** and **all features**, showing the **direction per bucket** and variability metrics (flip flags, range/std of mean SHAP, etc.).

> Change the `months` parameter below to control which sheets to include.


In [1]:
# === Parameters ===
from pathlib import Path

# Choose which workbook to read (pick one)
src_path = Path('Insights - Zoom - Marketing.xlsx')

# Which month sheets to include in the output (e.g., ['Full'] or ['January','February'])
# Use [] to auto-include all sheets from the workbook.
months = ['Full']

# Output Excel (single file; one sheet per month)
out_path = Path('zoomed_in_features/Zoom_Consistency_ByMonth.xlsx')

In [2]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

def normalize_bucket_order(buckets: pd.Series):
    def key(lbl):
        if pd.isna(lbl):
            return float('inf')
        s = str(lbl)
        # grab the first number in the label, e.g. "0 days to 4.23" -> 0, "6.46 days+" -> 6.46
        m = re.search(r"[-+]?\d*\.?\d+(?=\D|$)", s)
        return float(m.group()) if m else float('inf')
    return buckets.map(key)


def build_month_report(df_month: pd.DataFrame) -> pd.DataFrame:
    """Combine all anchors for a month into one tall table.
    Each row is a (anchor, feature), with bucket direction columns and metrics.
    """
    needed = {'Zoomed In Feature','bucket','feature','direction','mean_shap','mean_abs_shap'}
    missing = needed - set(df_month.columns)
    if missing:
        raise ValueError(f"Missing columns {missing}; did you run the latest Zoom export with direction columns?")

    # Bucket ordering
    ordered_buckets = (df_month[['bucket']].drop_duplicates()
                       .assign(_order=normalize_bucket_order(df_month['bucket'].drop_duplicates()))
                       .sort_values('_order')['bucket'].tolist())

    rows = []
    for anchor, dfa in df_month.groupby('Zoomed In Feature'):
        # pivot directions across buckets for this anchor
        pivot_dir = dfa.pivot_table(index='feature', columns='bucket', values='direction', aggfunc='first')
        pivot_dir = pivot_dir.reindex(columns=ordered_buckets)

        grp = dfa.groupby('feature')
        n_buckets_present = grp['bucket'].nunique()
        n_unique_dir = grp['direction'].nunique()

        def has_flip(series):
            vals = set(series.dropna().astype(str))
            return int(('Up' in vals) and ('Down' in vals))

        flip_flag = grp['direction'].apply(has_flip)
        std_mean = grp['mean_shap'].std(ddof=1)
        rng_mean = grp['mean_shap'].agg(lambda s: (s.max() - s.min()) if len(s)>0 else np.nan)
        avg_abs  = grp['mean_abs_shap'].mean()

        metrics = pd.DataFrame({
            'n_buckets_present': n_buckets_present,
            'n_unique_directions': n_unique_dir,
            'has_flip_Up_Down': flip_flag,
            'std_mean_shap': std_mean,
            'range_mean_shap': rng_mean,
            'avg_mean_abs_shap': avg_abs
        })

        report = pivot_dir.merge(metrics, left_index=True, right_index=True, how='left')
        # add anchor context
        report.insert(0, 'anchor', anchor)

        # consistency label
        def consistency_row(row):
            if row['has_flip_Up_Down'] == 1:
                return 'Variable (flip)'
            if row['n_unique_directions'] == 1:
                return 'Consistent'
            return 'Variable'
        report['consistency'] = report.apply(consistency_row, axis=1)

        # sort interesting first
        report = report.sort_values(
            by=['has_flip_Up_Down','range_mean_shap','std_mean_shap','avg_mean_abs_shap'],
            ascending=[False, False, False, False]
        ).reset_index().rename(columns={'index':'feature'})

        rows.append(report)

    # stack anchors into one sheet
    final = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    return final

In [3]:
# === Build the single Excel with one sheet per selected month ===
import pandas as pd
from pathlib import Path

assert src_path.exists(), f"Workbook not found: {src_path}"
xls = pd.ExcelFile(src_path)
all_sheets = xls.sheet_names

# If months list is empty, include all sheets
selected_months = months if months else all_sheets

# Build all selected month reports first
month_reports = {}
for m in selected_months:
    if m not in all_sheets:
        print(f"⚠️ Sheet '{m}' not found in {src_path.name}; skipping.")
        continue
    dfm = pd.read_excel(src_path, sheet_name=m)
    try:
        rep = build_month_report(dfm)
        month_reports[m] = rep
        print(f"✓ Built report for month: {m}  (rows={len(rep)})")
    except Exception as e:
        print(f"⚠️ Skipped '{m}' due to: {e}")

if not month_reports:
    raise RuntimeError('No month reports built. Check your `months` list and the workbook structure.')

# Write to one Excel (one sheet per month)
if out_path.exists():
    out_path.unlink()

with pd.ExcelWriter(out_path, engine='openpyxl', mode='w') as writer:
    # Optional: Index sheet
    idx = pd.DataFrame({
        'month': list(month_reports.keys()),
        'rows': [len(v) for v in month_reports.values()]
    })
    idx.to_excel(writer, sheet_name='Index', index=False)

    for m, rep in month_reports.items():
        sheet = m[:31]  # Excel limit
        rep.to_excel(writer, sheet_name=sheet, index=False)

print(f"✅ Wrote: {out_path.as_posix()}")

✓ Built report for month: Full  (rows=1740)
✅ Wrote: zoomed_in_features/Zoom_Consistency_ByMonth.xlsx


In [4]:
# QMVA Findings — centered images layout (v7, patched with signed value phrases + bold section headers)
# Output: /mnt/data/QMVA_Regression_Findings_Marketing_QUANT_v7.docx

from pathlib import Path
import pandas as pd, numpy as np, re
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# ---------- Paths ----------
BASE = Path("")
LOGO = BASE/"walker_logo.png"
INSIGHTS = BASE/"Insights.xlsx"
ZOOM = BASE/"Insights - Zoom - Marketing.xlsx"
SHAP_SUMMARY = BASE/"multivariate_modeling_results/shap/Full_SHAP_Summary_Plot.png"
ZOOM_DIR = BASE/"multivariate_modeling_results/shap_zoom/Full"
OUT = BASE/"QMVA_Regression_Findings_Marketing_QUANT_v7.docx"

MONTH = "Full"

# ---------- Utilities ----------
def prettify(s: str) -> str:
    if not isinstance(s,str): return str(s)
    s = re.sub(r"\s+"," ", s.replace("_"," ").strip())
    return " ".join(tok if (tok.isupper() and len(tok)<=6) else tok.capitalize() for tok in s.split())

def strip_units(lbl: str) -> str:
    return re.sub(r"\b(days?|hrs?|hours?)\b", "", str(lbl), flags=re.IGNORECASE).strip()

def normalize_bucket_order(df: pd.DataFrame):
    def key(lbl):
        if pd.isna(lbl): return float('inf')
        m = re.search(r"[-+]?\d*\.?\d+(?=\D|$)", str(lbl))
        return float(m.group()) if m else float('inf')
    return (df[["bucket"]].drop_duplicates()
            .assign(_o=lambda d: d["bucket"].map(key))
            .sort_values("_o"))["bucket"].tolist()

def set_justified_style(doc: Document):
    for nm in ['Normal','Heading 1','Heading 2','Heading 3']:
        try:
            st = doc.styles[nm]
            st.font.name = 'Calibri'
            st.font.size = Pt(11 if nm=='Normal' else (18 if nm=='Heading 1' else 14))
            if nm=='Normal':
                st.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
                st.paragraph_format.space_after = Pt(6)
        except KeyError:
            pass

def set_one_inch_margins(doc: Document):
    for sec in doc.sections:
        sec.top_margin = sec.bottom_margin = sec.left_margin = sec.right_margin = Inches(1)

def driver_strength_text(pos: int) -> str:
    if pos == 1: return "This is the highest driver in this period."
    if pos and pos <= 3: return "This is a strong driver in this period."
    if pos and pos <= 10: return "This feature is a moderate driver in this period."
    return "This feature is a minor driver in this period."

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "-", s.lower()).strip("-")

def _feature_name_variants(anchor: str):
    raw = anchor
    with_unders = anchor.replace(" ", "_")
    pretty = prettify(anchor).replace(" ", "_")
    no_dunders = re.sub(r"_+", "_", with_unders)
    return {raw, with_unders, pretty, no_dunders}

def find_zoom_image_for_anchor(anchor: str, zoom_dir: Path) -> Path | None:
    if not zoom_dir.exists(): return None
    files = list(zoom_dir.glob("*.png"))
    if not files: return None
    norm_to_path = { _norm(p.stem): p for p in files }
    for variant in _feature_name_variants(anchor):
        cand = f"Full_{variant}_ALL_BUCKETS_side_by_side"
        if _norm(cand) in norm_to_path:
            return norm_to_path[_norm(cand)]
    anchor_norm = _norm(anchor)
    candidates = [p for p in files if anchor_norm in _norm(p.stem)]
    def weight(p: Path):
        s = _norm(p.stem); score = 0
        if "all-buckets" in s: score += 3
        if "side-by-side" in s: score += 2
        if s.startswith("full-"): score += 1
        return score
    return sorted(candidates, key=weight, reverse=True)[0] if candidates else None

# ---------- NEW: signed value phrase helper ----------
def value_phrase_signed(feature_name: str, mean: float) -> str:
    """
    Return a short phrase that names the value that’s implied by the feature name,
    flipping to 'not <value>' when the mean SHAP is negative (i.e., that named value lowers).
    """
    f = feature_name.strip()

    # Gender M/F → flip to the opposite when mean < 0
    m = re.search(r'Consumer[_ ]?Gender[_ ]?([MF])$', f, flags=re.I)
    if m:
        val = m.group(1).upper()
        neg = 'F' if val == 'M' else 'M'
        return f"(when Gender = {val})" if mean >= 0 else f"(when Gender = {neg})"

    # *_Flag_Yes / *_Flag_No (treat 'Yes' as the named value)
    if re.search(r'(^|_)Flag($|_)', f, flags=re.I):
        if re.search(r'(^|_)Yes($|_)', f, flags=re.I):
            base = prettify(re.sub(r'[_ ]?Flag[_ ]?Yes', '', f, flags=re.I)).strip() or prettify(f)
            return f"(when {base} = Yes)" if mean >= 0 else f"(when {base} = No)"
        if re.search(r'(^|_)No($|_)', f, flags=re.I):
            base = prettify(re.sub(r'[_ ]?Flag[_ ]?No', '', f, flags=re.I)).strip() or prettify(f)
            return f"(when {base} = No)" if mean >= 0 else f"(when {base} = Yes)"

    # Terminal _Yes / _No
    m2 = re.search(r'(_Yes|_No)$', f, flags=re.I)
    if m2:
        base = prettify(re.sub(r'(_Yes|_No)$','', f, flags=re.I))
        val = m2.group(1).replace('_','').title()
        other = "No" if val == "Yes" else "Yes"
        return f"(when {base} = {val})" if mean >= 0 else f"(when {base} = {other})"

    # TimeOfDay_Night dummy
    if re.search(r'TimeOfDay[_ ]?Night$', f, flags=re.I):
        return "(when Night = Yes)" if mean >= 0 else "(when it is not Night)"

    # Season one-hots (… = Summer, flip to 'not Summer')
    sm = re.search(r'(Spring|Summer|Winter|Fall)$', f, flags=re.I)
    if sm and re.search(r'(Season|Call[_ ]?Season)', f, flags=re.I):
        season = sm.group(1).title()
        base = prettify(re.sub(r'(Spring|Summer|Winter|Fall)$','', f, flags=re.I)).strip()
        return f"(when {base} = {season})" if mean >= 0 else f"(when it is not {season})"

    # Geo/brand codes: treat suffix as the named category; flip to 'not <cat>' when mean < 0
    m3 = re.search(r'_([A-Z0-9]+(?:_[A-Z0-9]+)*)$', f)
    if m3 and len(m3.group(1)) <= 12:
        base = prettify(re.sub(r'_([A-Z0-9]+(?:_[A-Z0-9]+)*)$','', f))
        cat = m3.group(1).replace('_',' ')
        return f"(when {base} = {cat})" if mean >= 0 else f"(when not {cat})"

    # Fallback (continuous/unknown) — no side specified
    return ""

# ---------- Load data ----------
ins = pd.read_excel(INSIGHTS, sheet_name=MONTH)
zoom = pd.read_excel(ZOOM, sheet_name=MONTH)

# Rank ordering
if {"Zoomed In Feature","Zoomed In Feature Position"}.issubset(zoom.columns):
    rank_df = (zoom.groupby("Zoomed In Feature")["Zoomed In Feature Position"]
                 .min().reset_index()
                 .sort_values("Zoomed In Feature Position"))
else:
    rank_df = ins.sort_values("mean_abs_shap", ascending=False)[["feature"]].rename(columns={"feature":"Zoomed In Feature"})
    rank_df["Zoomed In Feature Position"] = range(1, len(rank_df)+1)

total_abs = float(ins["mean_abs_shap"].sum())
top5 = ins.sort_values("mean_abs_shap", ascending=False).head(5)[["feature","mean_abs_shap"]].copy()
top5["share"] = top5["mean_abs_shap"]/total_abs*100.0 if total_abs>0 else 0.0

def intro_for_feature(feat: str):
    row = ins[ins["feature"]==feat]
    if row.empty:
        row = ins[ins["feature"].str.replace(" ","_")==feat]
    if row.empty:
        return None
    r = row.iloc[0]
    share = (r["mean_abs_shap"]/total_abs*100.0) if total_abs>0 else np.nan
    pct_pos = float(r.get("pct_positive_shap", np.nan))*100.0 if not pd.isna(r.get("pct_positive_shap", np.nan)) else np.nan
    direction = "upward" if r["mean_shap"]>0 else ("downward" if r["mean_shap"]<0 else "balanced")
    return dict(mean_abs=float(r["mean_abs_shap"]),
                share=float(share) if share==share else np.nan,
                direction=direction,
                pct_pos=float(pct_pos) if pct_pos==pct_pos else np.nan)

# ---------- Bucket narratives ----------
def bucket_metrics(dfa: pd.DataFrame, anchor_feat: str):
    dfa = dfa.copy()
    if "direction" not in dfa.columns:
        eps = 1e-9
        dfa["direction"] = np.where(dfa["mean_shap"] > eps, "Up",
                            np.where(dfa["mean_shap"] < -eps, "Down", "Neutral"))
    tot_per_bucket = dfa.groupby("bucket")["n_rows"].max()
    ordered = normalize_bucket_order(dfa)
    sub = dfa[dfa["feature"] != anchor_feat]
    agg = (sub.groupby(["bucket", "feature"])
              .agg(mean_shap=("mean_shap", "mean"),
                   mean_abs_shap=("mean_abs_shap", "mean"),
                   n_rows=("n_rows", "max"),
                   direction=("direction", lambda s: s.mode().iat[0] if len(s.mode()) else "Neutral"))
              .reset_index())
    agg["bucket_rows"] = agg["bucket"].map(tot_per_bucket).astype(float)
    agg["bucket_share"] = np.where(agg["bucket_rows"] > 0, agg["n_rows"]/agg["bucket_rows"], np.nan)
    ordered_c = pd.Categorical(agg["bucket"], categories=ordered, ordered=True)
    agg = agg.assign(__bkey__=ordered_c).sort_values(["__bkey__", "mean_abs_shap"], ascending=[True, False]).drop(columns="__bkey__")
    return agg, ordered, tot_per_bucket

def bucket_level_narrative(dfa: pd.DataFrame, anchor_feat: str, top_k=3):
    agg, ordered, btot = bucket_metrics(dfa, anchor_feat)
    lines = []
    for b in ordered:
        slab = agg[agg["bucket"] == b]
        if slab.empty: continue
        b_label = strip_units(b)
        bucket_n = float(btot.get(b, np.nan) or np.nan)
        anchor_total = float(btot.sum())
        share = (bucket_n / anchor_total) if anchor_total > 0 else np.nan

        up = slab[slab["direction"] == "Up"].nlargest(top_k, "mean_abs_shap")[["feature", "mean_shap"]]
        dn = slab[slab["direction"] == "Down"].nlargest(top_k, "mean_abs_shap")[["feature", "mean_shap"]]

        def fmt_pair(feat, mean):
            phr = value_phrase_signed(str(feat), float(mean))
            phr = f" {phr}" if phr else ""
            return f"{prettify(feat)}{phr} (≈{mean:+.2f})"

        parts = []
        if len(up):
            parts.append(", ".join([fmt_pair(f, m) for f, m in up.itertuples(index=False)]) + " tend to increase the prediction")
        if len(dn):
            parts.append(", ".join([fmt_pair(f, m) for f, m in dn.itertuples(index=False)]) + " tend to decrease the prediction")

        lines.append(f"• In bucket ‘{b_label}’ (~{share*100:0.0f}% of this anchor’s data), " + " and ".join(parts) + ".") if parts \
             else lines.append(f"• In bucket ‘{b_label}’, effects are mixed; no clear pattern stands out.")
    return lines

def bucket_differences_narrative(dfa: pd.DataFrame, anchor_feat: str, max_items=8):
    agg, ordered, _ = bucket_metrics(dfa, anchor_feat)
    if agg.empty: return ["• No clear bucket-level differences found."]

    lines = []
    dir_piv = agg.pivot_table(index="feature", columns="bucket", values="direction", aggfunc="first").reindex(columns=ordered)
    val_piv = agg.pivot_table(index="feature", columns="bucket", values="mean_shap", aggfunc="mean").reindex(columns=ordered)

    def clean(lbl): return strip_units(lbl)

    flips = []
    for feat, row in dir_piv.iterrows():
        bnames = [b for b in row.index if pd.notna(row[b])]
        for i in range(len(bnames)-1):
            d1, d2 = row[bnames[i]], row[bnames[i+1]]
            if d1 in {"Up","Down"} and d2 in {"Up","Down"} and d1 != d2:
                v1, v2 = val_piv.loc[feat, bnames[i]], val_piv.loc[feat, bnames[i+1]]
                if pd.isna(v1) or pd.isna(v2): continue
                delta = abs(v2 - v1)
                denom = abs(v1) + abs(v2)
                pct_dev = (delta/denom*100.0) if denom > 1e-9 else np.nan
                flips.append((feat, bnames[i], bnames[i+1], d1, d2, delta, pct_dev))

    flips = sorted(flips, key=lambda t: t[5], reverse=True)[:max_items]
    for feat, b1, b2, d1, d2, delta, pct_dev in flips:
        pct_txt = f"{pct_dev:.0f}%" if pd.notna(pct_dev) else "—"
        # Use the sign of the *second* bucket to decide the phrase; either is fine since we describe both
        mean2 = val_piv.loc[feat, b2]
        phr = value_phrase_signed(str(feat), float(mean2))
        phr = f" {phr}" if phr else ""
        lines.append(
            f"• {prettify(feat)}{phr} increases in bucket ‘{clean(b1)}’ but decreases in bucket ‘{clean(b2)}’. "
            f"The difference is about {delta:.2f} SHAP points — roughly a {pct_txt} swing between these two buckets."
        )

    return lines or ["• No meaningful flips between buckets for this feature."]

# ---------- Layout helper ----------
def add_center_image(doc: Document, path: Path, width_inches=4.0):
    if path and path.exists():
        p = doc.add_paragraph(); p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        p.add_run().add_picture(str(path), width=Inches(width_inches))

# ---------- Build DOCX ----------
doc = Document()
set_one_inch_margins(doc)
set_justified_style(doc)

# Cover
if LOGO.exists():
    p = doc.add_paragraph(); p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    p = doc.add_paragraph(); p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    p.add_run().add_picture(str(LOGO), width=Inches(2.8))
p = doc.add_paragraph(); p.alignment = WD_ALIGN_PARAGRAPH.CENTER; p.style = doc.styles['Heading 1']
p.add_run("Regression Findings (Marketing)")
p = doc.add_paragraph(); p.alignment = WD_ALIGN_PARAGRAPH.CENTER; p.style = doc.styles['Heading 2']
p.add_run("Predicting QMVA — period overview and feature interactions")
doc.add_paragraph(f"Sources: {INSIGHTS.name} (overall), {ZOOM.name} (zoom buckets)")
doc.add_page_break()

# Period Overview
p = doc.add_paragraph(); p.style = doc.styles['Heading 1']; p.add_run("Period overview")
add_center_image(doc, SHAP_SUMMARY)
doc.add_paragraph(
    "Total SHAP impact this period reflects the combined importance of all features. "
    "Higher values mean the model’s predictions are driven more strongly by the inputs."
)
doc.add_paragraph(f"Overall total impact (sum of mean |SHAP| across features): approximately {total_abs:.2f}.")
doc.add_paragraph("Top contributors this period:")
for _, r in top5.iterrows():
    doc.add_paragraph(f"• {prettify(r['feature'])} — about {r['share']:.1f}% of total impact")
doc.add_page_break()

# Operational Definitions
p = doc.add_paragraph(); p.style = doc.styles['Heading 1']; p.add_run("Operational definitions")
defs = [
    ("SHAP value","Signed contribution: positive raises the prediction; negative lowers it."),
    ("Mean |SHAP| (impact)","Average absolute SHAP: overall influence, regardless of sign."),
    ("Direction (Up/Down)","Net tendency within a bucket: Up usually raises; Down lowers."),
    ("Bucket","A slice of the anchor feature’s range (e.g., ranges of Log Days Since Incident)."),
]
for t, d in defs: doc.add_paragraph(f"• {t}: {d}")
doc.add_page_break()

# Feature index
p = doc.add_paragraph(); p.style = doc.styles['Heading 1']; p.add_run("Feature index (this period)")
for _, rr in rank_df.iterrows():
    feat = str(rr["Zoomed In Feature"]); pos = int(rr["Zoomed In Feature Position"])
    doc.add_paragraph(f"Top {pos} — {prettify(feat)}")
doc.add_page_break()

# Detailed sections
for _, rr in rank_df.iterrows():
    anchor = str(rr["Zoomed In Feature"]); pos = int(rr["Zoomed In Feature Position"])

    h = doc.add_paragraph(); h.style = doc.styles['Heading 2']; h.add_run(f"{prettify(anchor)} — this period")
    doc.add_paragraph(driver_strength_text(pos))

    add_center_image(doc, find_zoom_image_for_anchor(anchor, ZOOM_DIR))

    st = intro_for_feature(anchor)
    if st:
        share_txt = f"{st['share']:.1f}%" if st['share']==st['share'] else "—"
        pos_txt = f"{st['pct_pos']:.0f}%" if st['pct_pos']==st['pct_pos'] else "—"
        doc.add_paragraph(
            f"Overall: this feature accounts for about {share_txt} of total model impact this period. "
            f"The net tendency is {st['direction']} (around {pos_txt} of cases push upward)."
        )
    else:
        doc.add_paragraph("Overall: impact summary available qualitatively; quantitative detail not found for this feature.")

    dfa = zoom[zoom["Zoomed In Feature"]==anchor].copy()
    if "direction" not in dfa.columns and "mean_shap" in dfa.columns:
        eps = 1e-9
        dfa["direction"] = np.where(dfa["mean_shap"]>eps,"Up",np.where(dfa["mean_shap"]<-eps,"Down","Neutral"))

    # --- Bold subsection header: per-bucket ---
    p = doc.add_paragraph(); run = p.add_run("How this feature behaves across buckets"); run.bold = True
    for line in bucket_level_narrative(dfa, anchor, top_k=3):
        doc.add_paragraph(line)

    # --- Bold subsection header: changes ---
    p = doc.add_paragraph(); run = p.add_run("Where behavior changes across buckets (with magnitude)"); run.bold = True
    for line in bucket_differences_narrative(dfa, anchor, max_items=8):
        doc.add_paragraph(line)

    doc.add_page_break()

doc.save(str(OUT))
print(str(OUT))


QMVA_Regression_Findings_Marketing_QUANT_v7.docx
